In [13]:
from amftrack.pipeline.development.high_mag_videos.kymo_class import *
import os
import pandas as pd
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
from glob import glob
from tqdm import tqdm
%matplotlib widget
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    clean_degree_4,
)
import scipy
from pathlib import Path
from amftrack.pipeline.launching.run_super import run_parallel
import re
from amftrack.util.dbx import upload_folders, upload, download, read_saved_dropbox_state, save_dropbox_state, load_dbx, get_dropbox_folders, get_dropbox_video_folders




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
directory_targ = "/gpfs/scratch1/shared/amftrackflow/Rachael_set/"
upload_targ = "/DATA/FLUORESCENCE/DATA_NileRed/"

In [3]:
test_name = glob(directory_targ + '*')[0]
imgs_address = test_name
print(test_name)

/gpfs/scratch1/shared/amftrackflow/Rachael_set/20230123_Plate545


In [37]:
plate_list = glob(directory_targ + "*_Plate*/*/")
datadict = {'address_total' : plate_list}
dataframe = pd.DataFrame(data=datadict)

print(dataframe['address_total'][0])

dataframe['video_name'] = [address[:-1].split('/')[-1] for address in dataframe['address_total']]
dataframe['plate_nr'] = [re.split('_|/', address.split('_')[-1])[0][5:] for address in dataframe['address_total']]
dataframe['video_nr'] = [int(re.split('_|/', address[:-1])[-1]) for address in dataframe['address_total']]
dataframe['date_imaged'] = [address.split('/')[-3][:8] for address in dataframe['address_total']]

# print(dataframe['date_imaged'])

parent_folder = []
data_table = []
magnification = []
fps = []
mode = []
binning = []

for row in dataframe.iloc:
#     print(row['address_total'])
    parent_folder.append(str(Path(row['address_total']).parent))
    excel_file = glob(str(Path(row['address_total']).parent) + f'/*{row["date_imaged"]}*{row["plate_nr"]}.xl*')
    if len(excel_file) > 0:
        data_table.append(excel_file[0])
        excel_table = pd.read_excel(excel_file[0])
#         print(row['video_name'])
#         print(excel_table['Unnamed: 0'][21])
#         print(excel_table[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['Magnification'])
        magnification.append(excel_table[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['Magnification'].iloc[0])
    
#         magnification.append(excel_table[excel_table['Unnamed: 0'].str.split(pat='_')[-1]] == row['video_name']['Magnification'].iloc[0])
    
        fps.append(excel_table.loc[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['FPS'].iloc[0])
        mode.append(excel_table.loc[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['Bright-field (BF)\nor\nFluorescence (F)'].iloc[0])
        if 'Binned (Y/N)' in excel_table:
            binning.append(excel_table.loc[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['Binned (Y/N)'].iloc[0])
        else:
            binning.append('N')
    else:
        csv_file = glob(str(Path(row['address_total']).parent) + f'/{row["date_imaged"]}*{row["plate_nr"]}.csv')
        if len(csv_file) > 0:
            data_table.append(csv_file[0])
            df_comma = pd.read_csv(csv_file[0], nrows=1,sep=",")
            df_semi = pd.read_csv(csv_file[0], nrows=1, sep=";")
            if df_comma.shape[1]>df_semi.shape[1]:
                csv_table = pd.read_csv(csv_file[0], sep=",")
            else:
                csv_table = pd.read_csv(csv_file[0], sep=";")
                
#             print(csv_table['video'])
#             print(row['video_name'])
            magnification.append(csv_table[csv_table['video']==int(row['video_name'])]['Lens'].iloc[0])
            fps.append(csv_table.loc[csv_table['video'] == int(row['video_name'].split("_")[-1])]['fps'].iloc[0])
#             print(csv_table.loc[csv_table['video'] == int(row['video_name'].split("_")[-1])]['fps'].iloc[0])
            mode.append(csv_table.loc[csv_table['video'] == int(row['video_name'].split("_")[-1])]['Illumination'].iloc[0])
        else:
            print("Halp! No datatable found! Pls add the excel file to the folders.")

dataframe['fps'] = fps
dataframe['magnification'] = magnification
dataframe['mode'] = mode
dataframe['parent_folder'] = [os.path.relpath(address, directory_targ) for address in dataframe['address_total']]
dataframe['data_table'] = data_table
dataframe['binned'] = binning

dataframe = dataframe.sort_values(by='video_name', ignore_index=True)
# dataframe = dataframe[dataframe['video_nr']== 9]
# dataframe = dataframe[dataframe['plate_nr'] == '046']
# print(len(dataframe))
# print(data_table)


if dataframe.isnull().values.any():
    print("Found NaNs in the excel files! Blame the experimentalists.")
    dataframe = dataframe.interpolate(method='pad', limit_direction='forward')
if dataframe.isnull().values.any():
    raise("This excel sheet is unworkable, please ask the responsible person")

/gpfs/scratch1/shared/amftrackflow/Rachael_set/20230123_Plate545/34/
Found NaNs in the excel files! Blame the experimentalists.


In [142]:
    
dataframe_filtered = dataframe[dataframe['video_name'] == '66']
dataframe_filtered = dataframe_filtered[dataframe_filtered['date_imaged'] == "20221026"]
dataframe_filtered['plate_nr'].unique()
dataframe_filtered

,address_total,video_name,plate_nr,video_nr,date_imaged,fps,magnification,mode,parent_folder,data_table,binned
782,/gpfs/scratch1/shared/amftrackflow/Rachael_set...,66,452,66,20221026,10.0,4.0,F,20221026_Plate452/66,/gpfs/scratch1/shared/amftrackflow/Rachael_set...,N


In [143]:
# FINAL CHECK FOR SOURCE FOLDER AND UPLOAD FOLDER

# Please make sure that the upload folder is correct, 
# as the program WILL overwrite that which is already there.
print(directory_targ)
print(upload_targ)

/gpfs/scratch1/shared/amftrackflow/Rachael_set/
/DATA/FLUORESCENCE/DATA_NileRed/


In [144]:
nr_parallel = np.min([len(dataframe.index), 16])

run_parallel(
    "flux_extract.py",
    [directory_targ, 15, 0.95, 0.001, 60, upload_targ],
    dataframe_filtered,
    nr_parallel,
    "1:00:00",
    "flux_extract"
)

/home/svstaalduine/bash/job.sh
Sending jobs with id 1687269230725776376
1


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2946081


Upload the proper excel files to the analysis folder.

In [11]:
excel_series = pd.Series(data_table).unique()
excel_series

array(['/gpfs/scratch1/shared/amftrackflow/Rachael_set/20230123_Plate545/Fluorescence_ex_20230123_Plate545.xlsx',
       '/gpfs/scratch1/shared/amftrackflow/Rachael_set/20221108_Plate462/Fluorescence_ex_20221108_plate462.xlsx',
       '/gpfs/scratch1/shared/amftrackflow/Rachael_set/20230125_Plate532/Fluorescence_ex_20230125_Plate532.xlsx',
       '/gpfs/scratch1/shared/amftrackflow/Rachael_set/20221109_Plate462/Fluorescence_ex_20221109_plate462.xlsx',
       '/gpfs/scratch1/shared/amftrackflow/Rachael_set/20230123_Plate530/Fluorescence_ex_20230123_Plate530.xlsx',
       '/gpfs/scratch1/shared/amftrackflow/Rachael_set/20230201_Plate552/Fluorescence_ex_20230201_Plate552.xlsx',
       '/gpfs/scratch1/shared/amftrackflow/Rachael_set/20221208_Plate510/Fluorescence_ex_20221208_Plate510.xlsx',
       '/gpfs/scratch1/shared/amftrackflow/Rachael_set/20221026_Plate452/Fluorescence_ex_20221026_plate452.xlsx',
       '/gpfs/scratch1/shared/amftrackflow/Rachael_set/20230201_Plate558/Fluorescence_ex

In [19]:
for xl_address in excel_series:
    file_name = "Analysis/" + os.path.relpath(xl_address, directory_targ)
    print(upload_targ + file_name)
    upload(xl_adress, upload_targ+file_name)

/DATA/FLUORESCENCE/DATA_NileRed/Analysis/20230123_Plate545/Fluorescence_ex_20230123_Plate545.xlsx
2023-06-20 11:12:22,949-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-20 11:12:23,475-[INFO]- dropbox:474 -> Request to files/upload
/DATA/FLUORESCENCE/DATA_NileRed/Analysis/20221108_Plate462/Fluorescence_ex_20221108_plate462.xlsx
2023-06-20 11:12:24,281-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-20 11:12:24,452-[INFO]- dropbox:474 -> Request to files/upload
/DATA/FLUORESCENCE/DATA_NileRed/Analysis/20230125_Plate532/Fluorescence_ex_20230125_Plate532.xlsx
2023-06-20 11:12:24,970-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-20 11:12:25,158-[INFO]- dropbox:474 -> Request to files/upload
/DATA/FLUORESCENCE/DATA_NileRed/Analysis/20221109_Plate462/Fluorescence_ex_20221109_plate462.xlsx
2023-06-20 11:12:25,825-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-20 11:12:26,029-[INFO]- dropbox:474 -> Request to files/upload
/DATA/FLUORESCENCE/DATA_NileRed/Anal